In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pickle
import numpy as np

/home1/arpitasa/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# file_path = '/home1/arpitasa/CSCI535/first_impressions_v2/transcription_test.pkl'
file_path = '/project/msoleyma_1026/personality_detection/first_impressions_v2_dataset/testing/transcription_test.pkl'
with open(file_path, 'rb') as file:
    data = pickle.load(file, encoding='latin1')

In [3]:
type(data)

dict

In [3]:

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

try:
    tokenizer =  AutoTokenizer.from_pretrained(MODEL)
    print("Tokenizer loaded successfully!")
except Exception as e:
    print(f"Error loading tokenizer: {e}")

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Tokenizer loaded successfully!


In [8]:
import os
save_folder = "/project/msoleyma_1026/personality_detection/first_impressions_v2_dataset/testing/text_feature_vectors"
train_dir = "/project/msoleyma_1026/personality_detection/first_impressions_v2_dataset/testing/video_files"

output_list = [] 
count = 0 

for train_fold in os.listdir(train_dir): 
    
    if not train_fold.startswith("test-") : 
        continue
    print(train_fold) 
    output_folder = os.path.join(save_folder, train_fold) 
    if not os.path.isdir(output_folder): 
        os.makedirs(output_folder)
    full_train_fold = os.path.join(train_dir, train_fold) 
    for sub_fold in os.listdir(full_train_fold): 
        print(sub_fold) 
        full_sub_fold = os.path.join(full_train_fold, sub_fold) 
        for vid_file in os.listdir(full_sub_fold): 
            full_vid_file = os.path.join(full_sub_fold,vid_file) 

            text = data[vid_file] 
            encoded_input = tokenizer(text, return_tensors='pt')
            model.config.output_hidden_states = True
            output = model(**encoded_input)
            scores = output[0][0].detach().numpy()
            scores = softmax(scores)

            ranking = np.argsort(scores)
            ranking = ranking[::-1]
            hidden_states = output.hidden_states 
            last_hidden_state = hidden_states[-1] 

            np_array = last_hidden_state[0].detach().numpy()
            averaged_vector = np.mean(np_array, axis=0)
            # print(np_array)
            output_list.append(averaged_vector)
            
            file_name = vid_file[:-4]
            # file_name = "temp"
            output_file = os.path.join(output_folder, file_name) 
            np.save(output_file, averaged_vector)
  

test-1
test80_13
test80_07
test80_10
test80_03
test80_09
test80_04
test80_06
test80_01
test80_12
test80_05
test80_08
test80_02
test80_11
test-2
test80_21
test80_19
test80_14
test80_22
test80_17
test80_25
test80_15
test80_20
test80_18
test80_16
test80_24
test80_23
